## Vocabulary Analysis

In [37]:
import transformers
from transformers import AutoModel, AutoTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
tokenizer.tokenize("CHUPAME BIEN LA PIJA PUTO")

['chupa', '##me', 'bien', 'la', 'pi', '##ja', 'puto']

In [39]:
tokenizer.tokenize("pija")

['pi', '##ja']

In [40]:
tokenizer.tokenize("trolo")

['tro', '##lo']

In [41]:
tokenizer.tokenize("maricón")

['maricón']

In [42]:
tokenizer.tokenize("marica")

['marica']

In [43]:
tokenizer.tokenize("puto")

['puto']

In [44]:
tokenizer.tokenize("Hacete ortear viejo trolazo")

['hace', '##te', 'or', '##tear', 'viejo', 'tro', '##laz', '##o']

Es un problema esto. Veamos cómo agregar posiblemente estos tokens...

## Training new tokenizer

In [45]:
from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets()

In [46]:
new_tokenizer.train_from_iterator?

Signature:
new_tokenizer.train_from_iterator(
    iterator: Union[Iterator[str], Iterator[Iterator[str]]],
    vocab_size: int = 30000,
    min_frequency: int = 2,
    limit_alphabet: int = 1000,
    initial_alphabet: List[str] = [],
    special_tokens: List[Union[str, tokenizers.AddedToken]] = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'],
    show_progress: bool = True,
    wordpieces_prefix: str = '##',
)
Docstring: Train the model using the given iterator 
File:      ~/.local/share/virtualenvs/hatespeech-classification-n4GdOxTz/lib/python3.8/site-packages/tokenizers/implementations/bert_wordpiece.py
Type:      method


Veamos los que están más de 5 veces

In [72]:
from tokenizers import BertWordPieceTokenizer

new_tokenizer = BertWordPieceTokenizer(lowercase=True)
texts = [ex["text"] for ex in train_dataset]
new_tokenizer.train_from_iterator(
    texts, min_frequency=10
)

In [73]:
new_tokenizer

Tokenizer(vocabulary_size=8939, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)

In [74]:
print(new_tokenizer.encode("Cría horrenda").tokens)

['cria', 'horr', '##enda']


In [75]:
old_tokens = set(tokenizer.get_vocab())

missing_tokens = [tok for tok in new_tokenizer.get_vocab() if tok not in old_tokens]

len(missing_tokens)

2906

Hay 5217 tokens que no están! Es un montón...

In [83]:
i = 400

for i, tok in enumerate(sorted(missing_tokens)):
    print(f"{i+1:<4} -- {tok}")

1    -- #
2    -- ##00000000
3    -- ##0000000000000000
4    -- ##5n
5    -- ##aaa
6    -- ##aaaa
7    -- ##aaaaa
8    -- ##aaaaaa
9    -- ##aaaaaaa
10   -- ##aaaaaaaa
11   -- ##aaaaaaaaa
12   -- ##abamos
13   -- ##abon
14   -- ##acer
15   -- ##aces
16   -- ##acha
17   -- ##ache
18   -- ##acho
19   -- ##acia
20   -- ##acio
21   -- ##acion
22   -- ##acios
23   -- ##acter
24   -- ##acto
25   -- ##adito
26   -- ##aj
27   -- ##aja
28   -- ##ajaa
29   -- ##ajaj
30   -- ##ajaja
31   -- ##ajajaj
32   -- ##ajajaja
33   -- ##ajajajaj
34   -- ##ajajajaja
35   -- ##ajajajajajajajaj
36   -- ##ajajajajajajajajajajajajajajajaj
37   -- ##ajajajajajajajajajajajajajajajajajajajajajajajajajajajajajajajaj
38   -- ##ajajajajj
39   -- ##ajajajj
40   -- ##ajajajjajajajj
41   -- ##ajajj
42   -- ##ajas
43   -- ##ajj
44   -- ##ajja
45   -- ##ajjaja
46   -- ##ajo
47   -- ##aju
48   -- ##ajuaju
49   -- ##ajud
50   -- ##ajudicial
51   -- ##alberto
52   -- ##alde
53   -- ##aleza
54   -- ##alimos
55   -- ##altad
56

Hay que preprocesar esto, es una cagada así

Primero, emojis

In [84]:
import emoji

emoji.demojize("Viva Perón ✌✌✌✌✌✌✌✌✌✌✌")

'Viva Perón :victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand::victory_hand:'

In [124]:
import re
hashtag_regex = re.compile(r'\B#(\w*[a-zA-Z]+\w*)')

text = "Probando #PreprocesamientoDeHashtags a #versi anda bien #"
hashtag_regex.findall(text)

['PreprocesamientoDeHashtags', 'versi']

In [125]:
interspace = re.compile(r'([A-Z]+)')

interspace.sub(r' \1', "CamelCaseSarasa").lower().strip()

'camel case sarasa'

In [126]:

def sarasa(x):
    print(x)
    text = x.groups()[0]
    
    return camel_to_human(text)



hashtag_regex.sub(
    sarasa,
    text
)




<re.Match object; span=(9, 36), match='#PreprocesamientoDeHashtags'>
<re.Match object; span=(39, 45), match='#versi'>


'Probando Preprocesamiento De Hashtags a versi anda bien #'